In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as T
import torchvision.models
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import numpy as np
import pandas as pd
from PIL import Image
from sklearn import preprocessing, metrics, model_selection

In [3]:
USE_GPU = True

dtype = torch.float # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [4]:
train_mean = 0.0
train_std = 0.0
val_mean = 0.0
val_std = 0.0

In [7]:
# load images
from os import listdir, path

images_list = listdir('/content/drive/MyDrive/SC201_Final_Project/images/map')

print(images_list)

image_dic_index = {}
image_dic_latlng = {}
for i in range(len(images_list)):
  image_dic_index[i] = images_list[i][:len(images_list[i])-4]
  image_dic_latlng[images_list[i][:len(images_list[i])-4]] = i
print(image_dic_index)
print(image_dic_latlng)

['25.07167_121.36408.png', '24.94512_121.37715.png', '25.00218_121.5108.png', '25.18119_121.42169.png', '25.05444_121.43371.png', '25.17048_121.44223.png', '25.06271_121.49759.png', '24.99944_121.50738.png', '25.00441_121.4343.png', '25.0558_121.43412.png', '25.00609_121.45888.png', '25.05728_121.44461.png', '25.08543_121.46053.png', '25.13007_121.46284.png', '25.18296_121.41956.png', '24.94744_121.37932.png', '25.18303_121.41955.png', '25.18134_121.42035.png', '25.08449_121.46233.png', '25.00667_121.51038.png', '25.00512_121.51365.png', '24.93428_121.37117.png', '24.98202_121.45743.png', '25.07648_121.37312.png', '25.00866_121.50909.png', '25.08976_121.44749.png', '24.94788_121.38104.png', '24.94596_121.37944.png', '24.99968_121.50347.png', '25.0456_121.45304.png', '25.02812_121.47324.png', '24.9463171_121.3757444.png', '24.996_121.46272.png', '25.08451_121.47165.png', '25.07335_121.66225.png', '25.05506_121.43284.png', '25.00017_121.43251.png', '25.00438_121.43444.png', '25.0585_121.

In [8]:
nan_cache = {}
def data_preprocess(filename,mode='Train'):
  data = pd.read_csv(filename)
  data.insert(len(data.columns), 'index', '')
  data.loc[data['lat_lng'] == '25_121.50227','lat_lng'] = '25.0_121.50227'
  index_list = []
  for index, row in data.iterrows():
    index_list.append(image_dic_latlng[row[2]])
  data['index'] = index_list
  if mode == 'Train':
    # split data to train, validation and test datsets
    train_data, val_data = model_selection.train_test_split(data,test_size = 0.2)
    means = train_data.mean() # to use mean or other indicators to fillna?
    train_data = train_data.fillna(means)
    val_data = val_data.fillna(means)
    for column in data:
      nan_cache[column] = means
    train_y = train_data.pop('price_30000')
    val_y = val_data.pop('price_30000')
    return train_data, val_data, train_y, val_y
  elif mode == 'Test':
    real_test_data = data.fillna(nan_cache)
    return real_test_data

In [9]:
district_list = ['villages_towns_Sanxia', 'villages_towns_Sanzhi',
       'villages_towns_Sanchong', 'villages_towns_Zhonghe',
       'villages_towns_Zhongshan', 'villages_towns_Zhongzheng',
       'villages_towns_Wugu', 'villages_towns_Xinyi', 'villages_towns_Neihu',
       'villages_towns_Bali', 'villages_towns_Beitou',
       'villages_towns_Nangang', 'villages_towns_Tucheng',
       'villages_towns_Shilin', 'villages_towns_Datong', 'villages_towns_Daan',
       'villages_towns_Wenshan', 'villages_towns_Xindian',
       'villages_towns_Xinzhuang', 'villages_towns_Songshan',
       'villages_towns_Banqiao', 'villages_towns_Linkou',
       'villages_towns_Shulin', 'villages_towns_Yonghe',
       'villages_towns_Xizhi', 'villages_towns_Taishan',
       'villages_towns_Tamsui', 'villages_towns_Shenkeng',
       'villages_towns_Ruifang', 'villages_towns_Wanhua',
       'villages_towns_Wanli', 'villages_towns_Luzhou',
       'villages_towns_Gongliao', 'villages_towns_Jinshan',
       'villages_towns_Shuangxi', 'villages_towns_Yingge']
building_material = ['building_materials_RC', 'building_materials_RB',
       'building_materials_brick', 'building_materials_steel',
       'building_materials_SRC', 'building_materials_PRX',
       'building_materials_other_material']

In [10]:
FILE = '/content/drive/MyDrive/SC201_Final_Project/Data/new_data_0902.csv'

In [11]:
train_data, val_data, train_y, val_y = data_preprocess(FILE,mode='Train')
print(val_data.shape)


(5534, 93)


In [ ]:
train_data.head()

,lat,lng,lat_lng,zoning,total_floors,floors_area,room,hall,bathroom,compartmented,management_committee,unit_price,price_15000,unit_berth_price,total_berth_price,main_building_area,auxiliary_building_area,balcony_area,elevator,building_age,travel_agency,point_of_interest,establishment,clothing_store,home_goods_store,store,local_government_office,university,natural_feature,health,tourist_attraction,transit_station,food,transaction_first_floor,shifting_level,villages_towns_Sanxia,villages_towns_Sanzhi,villages_towns_Sanchong,villages_towns_Zhonghe,villages_towns_Zhongshan,...,villages_towns_Xinzhuang,villages_towns_Songshan,villages_towns_Banqiao,villages_towns_Linkou,villages_towns_Shulin,villages_towns_Yonghe,villages_towns_Xizhi,villages_towns_Taishan,villages_towns_Tamsui,villages_towns_Shenkeng,villages_towns_Ruifang,villages_towns_Wanhua,villages_towns_Wanli,villages_towns_Luzhou,villages_towns_Gongliao,villages_towns_Jinshan,villages_towns_Shuangxi,villages_towns_Yingge,transaction_type_B,transaction_type_BL,transaction_type_BLP,building_state_residential building,building_state_elevator_apartment,building_state_apartment,building_state_studio,building_state_single_house,building_state_commercial_building,building_state_store,building_state_factory_office,building_state_warehouse,building_state_other_state,building_materials_SC,building_materials_RC,building_materials_RB,building_materials_brick,building_materials_steel,building_materials_SRC,building_materials_PRX,building_materials_other_material,index
25075,25.04968,121.52658,25.04968_121.52658,-0.333311,-1.239790,-0.464046,1,1,1,1,1,0.381062,12,-0.258582,-0.237700,-0.639499,-0.271472,-0.567518,1.0,1.192376,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1252
23821,24.98310,121.46582,24.9831_121.46582,-0.105931,0.896809,-0.212357,2,2,1,1,0,-0.259067,8,-0.258582,-0.237700,-0.276331,0.074599,-0.300513,1.0,-0.940053,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2176
15553,25.03205,121.55786,25.03205_121.55786,-0.417083,-0.305028,-0.430848,1,1,1,1,1,1.024042,15,-0.258582,-0.237700,-0.570946,-0.318818,-0.622907,1.0,0.192800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,3684
13237,25.04558,121.50664,25.04558_121.50664,-0.220818,-0.839178,-0.241336,2,2,2,1,1,-0.445439,7,-0.258582,-0.237700,-0.121270,-0.318818,-1.010632,1.0,1.792122,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,875
27319,25.18255,121.41963,25.18255_121.41963,-0.232785,1.698034,0.006822,2,1,1,1,1,-0.551117,7,0.160046,0.182634,-0.172685,0.056563,0.094313,1.0,-0.806776,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,514


In [ ]:
val_data.head()

,lat,lng,lat_lng,zoning,total_floors,floors_area,room,hall,bathroom,compartmented,management_committee,unit_price,price_15000,unit_berth_price,total_berth_price,main_building_area,auxiliary_building_area,balcony_area,elevator,building_age,travel_agency,point_of_interest,establishment,clothing_store,home_goods_store,store,local_government_office,university,natural_feature,health,tourist_attraction,transit_station,food,transaction_first_floor,shifting_level,villages_towns_Sanxia,villages_towns_Sanzhi,villages_towns_Sanchong,villages_towns_Zhonghe,villages_towns_Zhongshan,...,villages_towns_Xinzhuang,villages_towns_Songshan,villages_towns_Banqiao,villages_towns_Linkou,villages_towns_Shulin,villages_towns_Yonghe,villages_towns_Xizhi,villages_towns_Taishan,villages_towns_Tamsui,villages_towns_Shenkeng,villages_towns_Ruifang,villages_towns_Wanhua,villages_towns_Wanli,villages_towns_Luzhou,villages_towns_Gongliao,villages_towns_Jinshan,villages_towns_Shuangxi,villages_towns_Yingge,transaction_type_B,transaction_type_BL,transaction_type_BLP,building_state_residential building,building_state_elevator_apartment,building_state_apartment,building_state_studio,building_state_single_house,building_state_commercial_building,building_state_store,building_state_factory_office,building_state_warehouse,building_state_other_state,building_materials_SC,building_materials_RC,building_materials_RB,building_materials_brick,building_materials_steel,building_materials_SRC,building_materials_PRX,building_materials_other_material,index
21630,24.97864,121.44306,24.97864_121.44306,0.760109,1.163884,0.655923,4,2,2,1,1,-0.276771,8,1.400032,1.686134,0.333885,0.285398,0.643946,1.0,-0.806776,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2227
12683,25.05769,121.44806,25.05769_121.44806,-0.323737,1.030347,-0.361014,0,0,0,0,1,-0.164410,9,-0.258582,-0.237700,-0.502393,-0.144091,-0.370105,1.0,-0.806776,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,558
12450,24.92870,121.37642,24.9287_121.37642,-0.154997,-0.305028,-0.108682,3,1,1,1,1,-1.206380,3,-0.258582,-0.237700,-0.065775,-0.318818,0.331493,1.0,0.725907,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3551
14003,23.48357,120.44058,23.48357_120.44058,0.314523,-0.705640,0.063681,3,2,2,1,1,1.716851,19,-0.258582,-0.237700,0.026212,0.887359,-0.015045,1.0,-0.806776,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4157
640,25.07289,121.36749,25.07289_121.36749,-0.193293,1.030347,0.013609,2,2,2,1,1,-0.280106,8,0.132993,0.311967,-0.136776,0.134344,0.317291,1.0,-0.873415,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1909


In [12]:
# standardize data
train_mean = 154170.694
train_std = 79570.40139
to_drop = ['unit_price','price_15000','zoning', 'lat','lng','lat_lng','unit_berth_price','compartmented','management_committee','floors_area','establishment','clothing_store','home_goods_store','store','local_government_office','university','natural_feature','health','tourist_attraction','transit_station','food']
for drop_items in to_drop:
  train_data.pop(drop_items)
  val_data.pop(drop_items)

print(train_data.columns)
print(val_data.shape)

Index(['total_floors', 'room', 'hall', 'bathroom', 'total_berth_price',
       'main_building_area', 'auxiliary_building_area', 'balcony_area',
       'elevator', 'building_age', 'travel_agency', 'point_of_interest',
       'transaction_first_floor', 'shifting_level', 'villages_towns_Sanxia',
       'villages_towns_Sanzhi', 'villages_towns_Sanchong',
       'villages_towns_Zhonghe', 'villages_towns_Zhongshan',
       'villages_towns_Zhongzheng', 'villages_towns_Wugu',
       'villages_towns_Xinyi', 'villages_towns_Neihu', 'villages_towns_Bali',
       'villages_towns_Beitou', 'villages_towns_Nangang',
       'villages_towns_Tucheng', 'villages_towns_Shilin',
       'villages_towns_Datong', 'villages_towns_Daan',
       'villages_towns_Wenshan', 'villages_towns_Xindian',
       'villages_towns_Xinzhuang', 'villages_towns_Songshan',
       'villages_towns_Banqiao', 'villages_towns_Linkou',
       'villages_towns_Shulin', 'villages_towns_Yonghe',
       'villages_towns_Xizhi', 'villages_t

In [ ]:
val_data.head()

,total_floors,room,hall,bathroom,total_berth_price,main_building_area,auxiliary_building_area,balcony_area,elevator,building_age,travel_agency,point_of_interest,transaction_first_floor,shifting_level,villages_towns_Sanxia,villages_towns_Sanzhi,villages_towns_Sanchong,villages_towns_Zhonghe,villages_towns_Zhongshan,villages_towns_Zhongzheng,villages_towns_Wugu,villages_towns_Xinyi,villages_towns_Neihu,villages_towns_Bali,villages_towns_Beitou,villages_towns_Nangang,villages_towns_Tucheng,villages_towns_Shilin,villages_towns_Datong,villages_towns_Daan,villages_towns_Wenshan,villages_towns_Xindian,villages_towns_Xinzhuang,villages_towns_Songshan,villages_towns_Banqiao,villages_towns_Linkou,villages_towns_Shulin,villages_towns_Yonghe,villages_towns_Xizhi,villages_towns_Taishan,villages_towns_Tamsui,villages_towns_Shenkeng,villages_towns_Ruifang,villages_towns_Wanhua,villages_towns_Wanli,villages_towns_Luzhou,villages_towns_Gongliao,villages_towns_Jinshan,villages_towns_Shuangxi,villages_towns_Yingge,transaction_type_B,transaction_type_BL,transaction_type_BLP,building_state_residential building,building_state_elevator_apartment,building_state_apartment,building_state_studio,building_state_single_house,building_state_commercial_building,building_state_store,building_state_factory_office,building_state_warehouse,building_state_other_state,building_materials_SC,building_materials_RC,building_materials_RB,building_materials_brick,building_materials_steel,building_materials_SRC,building_materials_PRX,building_materials_other_material,index
21630,1.163884,4,2,2,1.686134,0.333885,0.285398,0.643946,1.0,-0.806776,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2227
12683,1.030347,0,0,0,-0.237700,-0.502393,-0.144091,-0.370105,1.0,-0.806776,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,558
12450,-0.305028,3,1,1,-0.237700,-0.065775,-0.318818,0.331493,1.0,0.725907,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3551
14003,-0.705640,3,2,2,-0.237700,0.026212,0.887359,-0.015045,1.0,-0.806776,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,4157
640,1.030347,2,2,2,0.311967,-0.136776,0.134344,0.317291,1.0,-0.873415,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1909


In [13]:
# load images to dictionary
def load_imgs(lat_lng):
  image = Image.open('/content/drive/MyDrive/SC201_Final_Project/images/map/{}.png'.format(lat_lng)).convert('RGB')
  image = T.ToTensor()(image)
  return image

In [14]:
class HousingDataset(torch.utils.data.Dataset):
  def __init__(self,dataset,labels):
    self.y_train = torch.tensor(labels.values,dtype=torch.float32).view(len(labels),1)
    self.x_train = torch.tensor(dataset.values,dtype=torch.float32)
    print("HousingDataset shape {}, {}".format(self.x_train.shape, self.y_train.shape))
  def __len__(self):
    return len(self.y_train)
  def __getitem__(self,idx):
    index = int(self.x_train[idx][71])
    # print(index)      # how to extract values from tensor
    x_img = load_imgs(image_dic_index[index])
    return self.x_train[idx], x_img , self.y_train[idx]

In [15]:
# Datasets load training examples one at a time, so we wrap each Dataset in a 
# DataLoader which iterates through the Dataset and forms minibatches. We divide
# the training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
# This is for after map data is loaded into the features, so shuffling is done
BATCH_SIZE = 32
def form_minibatch(data,mode='Train'):
  if mode == 'Train':
    data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)
  elif mode == 'Test':
    data = DataLoader(data,batch_size=BATCH_SIZE)
  return data

train_dl = form_minibatch(HousingDataset(train_data,train_y))
eval_dl = form_minibatch(HousingDataset(val_data,val_y),mode='Test')
print("training size {}, eval size {}".format(len(train_dl), len(eval_dl)))

HousingDataset shape torch.Size([22132, 72]), torch.Size([22132, 1])
HousingDataset shape torch.Size([5534, 72]), torch.Size([5534, 1])
training size 692, eval size 173


In [33]:
# model = MyModel()
class MyModel(nn.Module):
    def __init__(self,input_size,output_size):
        super().__init__()
        self.nn = nn.Sequential(
        nn.Linear(input_size, 60),
        nn.BatchNorm1d(60),
        nn.ReLU(),
        nn.Linear(60, 45),
        nn.BatchNorm1d(45),
        nn.ReLU(),
        nn.Linear(45, output_size),
        nn.BatchNorm1d(output_size),
        # nn.ReLU(),
        # nn.Linear(15, output_size),
        # nn.BatchNorm1d(output_size),
        )
        # epoch 25-29
        resnet = torchvision.models.resnet50(pretrained=True)
        for param in resnet.parameters():
          param.requires_grad = True
        

        self.features2 = nn.Sequential(
              nn.AdaptiveAvgPool2d((224,224)),
              *list(resnet.children())[:6],

              # N x 2048 * 28 * 28
              # nn.Conv2d(in_channels=2048, out_channels=1024, kernel_size=3, padding=1),
              # nn.BatchNorm2d(1024),
              # nn.ReLU(),

              # # N x 1024 * 28 * 28
              # nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=3, padding=1),
              # nn.BatchNorm2d(1024),
              # nn.ReLU(),

              # N x 512 * 28 * 28
              nn.Conv2d(in_channels=512, out_channels=256, kernel_size=3, padding=1),
              nn.BatchNorm2d(256),
              nn.ReLU(),

              # N x 256 * 28 * 28
              nn.Conv2d(in_channels=256, out_channels=128, kernel_size=3, padding=1),
              nn.BatchNorm2d(128),
              nn.ReLU(),

              # N x 128 * 28 * 28
              nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, padding=1),
              nn.BatchNorm2d(64),
              nn.ReLU(),

              # N x 64 x 28 x 28  
              nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, padding=1),
              nn.BatchNorm2d(32),
              nn.ReLU(),

              nn.MaxPool2d(kernel_size=2, stride=2),  

              # N x 32 x 14 x 14
              nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, padding=1),
              nn.BatchNorm2d(16),
              nn.ReLU(),

              # N x 16 x 14 x 14
              nn.Conv2d(in_channels=16, out_channels=8, kernel_size=3, padding=1),
              nn.BatchNorm2d(8),
              nn.ReLU(),

              nn.MaxPool2d(kernel_size=2, stride=2),

              # N x 8 x 7 x 7
              
              nn.Flatten(),
              # nn.Linear(2048,8)
              # nn.Linear(2048, 256),
              # nn.BatchNorm1d(256),
              # nn.ReLU(),
              nn.Linear(8*7*7,output_size),
              # nn.BatchNorm1d(16),

          )
        self.metrics = 0
        self.fc_out = nn.Sequential(
            nn.Linear(output_size*2,output_size)
            # nn.BatchNorm1d(output_size),
            # nn.ReLU(),
            # nn.Linear(output_size,1)
            # nn.BatchNorm1d(16),
            # nn.ReLU(),
            # nn.Linear(16, 8),
            # nn.BatchNorm1d(8),
            # nn.ReLU(),
            # nn.Linear(8, 4),
            # nn.BatchNorm1d(4),
            # nn.ReLU(),
            # nn.Linear(4,1)
        )


    def forward(self,x1,x2):
        x1 = self.nn(x1)
        x1 = x1.view(x1.size(0), -1)
        x1 = F.relu(x1)

        x2 = self.features2(x2)
        x2 = x2.view(x2.size(0), -1)
        x2 = F.relu(x2)
        # Concatenate in dim1 (feature dimension)
        x = torch.cat((x1, x2),1)
        scores = self.fc_out(x)
        return scores

net = MyModel(71,33)
optimizer = torch.optim.Adam(net.parameters(),lr=1e-4,weight_decay=0.99)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10)
optimizer_SGD_momentum = torch.optim.SGD(net.parameters(),lr=1e-5,momentum=0.9,weight_decay=0.98)

In [34]:
loss_history=[]
def train(data1, data2, model, optimizer, epochs=1):
    """
    Train a model on real estate data property features 
    and extracte map features using the PyTorch Module API.
    
    Inputs:
    - data1: training data
    - data2: validation data
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x,x2,y) in enumerate(data1):
        # for t, (x1,y) in enumerate(data1):
            model.train()  # put model to training mode
            indices = torch.tensor(range(0, 71))    # how to drop column from tensor
            x1 = torch.index_select(x, 1, indices)
            x1 = x1.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            # print(x1.size())
            x2 = x2.to(device=device,dtype=dtype)
            y = y.to(device=device, dtype=torch.long)
            
            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()
            
            scores = model(x1,x2)
            # prediction = scores.data.max(1)[1]
            # print(scores)
            # print(scores.size())
            # print(y.size())
            # scores = torch.squeeze(scores)
            y = torch.squeeze(y)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(scores,y)
            # running_loss += loss.item()
            # loss_history.append(loss)
            # running_loss += loss.item()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()
            # training_loss = running_loss/len(data1)
            if t % 100 == 0:
                # running_loss = 0.0
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                print('Checking accuracy on validation set')
                num_correct, num_samples, acc = check_accuracy(data2, model)
                print('Epoch %d, Got %d / %d correct (%.2f)' % (e,num_correct, num_samples, acc))
        lr_scheduler.step(model.metrics)


In [35]:
def check_accuracy(data, model):
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
      for x,x2,y in data:
      # for x1,y in data:
        indices = torch.tensor(range(0, 71))
        x1 = torch.index_select(x, 1, indices)
        # print(x1.size())
        x1 = x1.to(device=device, dtype=dtype)  # move to device, e.g. GPU
        x2 = x2.to(device=device,dtype=dtype)
        y = y.to(device=device, dtype=torch.long)
        scores = model(x1,x2)
        y = torch.squeeze(y)
        # scores = model(x1)
        _, preds = scores.max(1)
        num_correct += (preds == y).sum()
        num_samples += preds.size(0)
      acc = float(num_correct) / num_samples
      return num_correct, num_samples, 100*acc

In [36]:
train(train_dl, eval_dl, net,optimizer,epochs=28)

Iteration 0, loss = 3.6507
Checking accuracy on validation set
Epoch 0, Got 2 / 5534 correct (0.04)
Iteration 100, loss = 2.9917
Checking accuracy on validation set
Epoch 0, Got 1535 / 5534 correct (27.74)
Iteration 200, loss = 2.5331
Checking accuracy on validation set
Epoch 0, Got 1659 / 5534 correct (29.98)
Iteration 300, loss = 2.3670
Checking accuracy on validation set
Epoch 0, Got 1759 / 5534 correct (31.79)
Iteration 400, loss = 2.2266
Checking accuracy on validation set
Epoch 0, Got 1890 / 5534 correct (34.15)
Iteration 500, loss = 2.2509
Checking accuracy on validation set
Epoch 0, Got 2003 / 5534 correct (36.19)
Iteration 600, loss = 2.2794
Checking accuracy on validation set
Epoch 0, Got 1796 / 5534 correct (32.45)
Iteration 0, loss = 1.9146
Checking accuracy on validation set
Epoch 1, Got 1961 / 5534 correct (35.44)
Iteration 100, loss = 1.9768
Checking accuracy on validation set
Epoch 1, Got 1500 / 5534 correct (27.11)
Iteration 200, loss = 2.1105
Checking accuracy on vali

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt

plt.plot(range(len(loss_history)),loss_history)
plt.xlabel('iteration')
plt.ylabel('training loss')
plt.title('Training Loss history')
plt.show()

In [ ]:
# def predict(data,model):
#     model.eval()  # set model to evaluation mode
#     data = data_preprocess(data,mode='Test')
#     data = standardize(data)
#     data = data.to_numpy()
#     data = torch.from_numpy(data)
#     loader_data = form_minibatch(data,mode='Test')
#     with torch.no_grad():
#       for x1,x2 in loader_data:
#         x1 = x1.to(device=device, dtype=dtype)  # move to device, e.g. GPU
#         x2 = x2.to(device=device, dtype=dtype)
#         scores = model(x1,x2)
#     return scores

In [ ]:
# sort image according to train_data
lat_data = train_data['lat']
lng_data = train_data['lng']
for i in range(len(lat_data)):
  if i == 0:
    temp_lat = lat_data.iloc[i]
    temp_lng = lng_data.iloc[i]
    if f'{temp_lat}_{temp_lng}' in image_dic:
      sorted_imgs = image_dic[f'{temp_lat}_{temp_lng}']
    elif f'{round(temp_lat,3)}_{round(temp_lng,3)}' in image_dic:
      sorted_imgs = image_dic[f'{round(temp_lat,3)}_{round(temp_lng,3)}']
    elif f'{round(temp_lat,4)}_{round(temp_lng,4)}' in image_dic:
      sorted_imgs = image_dic[f'{round(temp_lat,4)}_{round(temp_lng,4)}']
    elif f'{round(temp_lat,5)}_{round(temp_lng,5)}' in image_dic:
      sorted_imgs = image_dic[f'{round(temp_lat,5)}_{round(temp_lng,5)}']
    elif f'{round(temp_lat,6)}_{round(temp_lng,6)}' in image_dic:
      sorted_imgs = image_dic[f'{round(temp_lat,6)}_{round(temp_lng,6)}']
    elif f'{round(temp_lat,7)}_{round(temp_lng,7)}' in image_dic:
      sorted_imgs = image_dic[f'{round(temp_lat,7)}_{round(temp_lng,7)}']
    print(sorted_imgs.size())
  else:
    temp_lat = lat_data.iloc[i]
    temp_lng = lng_data.iloc[i]
    if f'{temp_lat}_{temp_lng}' in image_dic:
      sorted_imgs = torch.stack((sorted_imgs,image_dic[f'{temp_lat}_{temp_lng}']),0)
    elif f'{round(temp_lat,3)}_{round(temp_lng,3)}' in image_dic:
      sorted_imgs = torch.stack((sorted_imgs,image_dic[f'{round(temp_lat,3)}_{round(temp_lng,3)}']),0)
    elif f'{round(temp_lat,4)}_{round(temp_lng,4)}' in image_dic:
      sorted_imgs = torch.stack((sorted_imgs,image_dic[f'{round(temp_lat,4)}_{round(temp_lng,4)}']),0)
    elif f'{round(temp_lat,5)}_{round(temp_lng,5)}' in image_dic:
      sorted_imgs = torch.stack((sorted_imgs,image_dic[f'{round(temp_lat,5)}_{round(temp_lng,5)}']),0)
    elif f'{round(temp_lat,6)}_{round(temp_lng,6)}' in image_dic:
      sorted_imgs = torch.stack((sorted_imgs,image_dic[f'{round(temp_lat,6)}_{round(temp_lng,6)}']),0)
    elif f'{round(temp_lat,7)}_{round(temp_lng,7)}' in image_dic:
      sorted_imgs = torch.stack((sorted_imgs,image_dic[f'{round(temp_lat,7)}_{round(temp_lng,7)}']),0)
    print(sorted_imgs.size())

NameError: ignored